In [8]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer
import torch

hub_name = "StarkWizard/Mistral-7b-instruct-cairo-instruct"
quant_name = hub_name + "-AWQ"

# The model that you want to train from the Hugging Face hub
model_name = "mistralai/Mistral-7B-Instruct-v0.1"

# The instruction dataset to use
dataset_name = "StarkWizard/cairo-instruct"

quant_config = {
    "zero_point":True,
    "q_group_size":128,
    "w_bit":4,
}

In [ ]:
#Loading Model and tokenizer

model = AutoAWQForCausalLM.from_pretrained(hub_name,device_map="auto",)



In [11]:
tokenizer = AutoTokenizer.from_pretrained(hub_name,trust_remote_code=True, use_auth_token=True, add_eos_token=True, use_fast=False)
tokenizer.padding_side = "right"
tokenizer.pad_token_id = 18610

#quantize
model.quantize(tokenizer=tokenizer,quant_config=quant_config)

#Now we save
model.save_quantized(quant_name, safetensors=True, shard_size="1GB")
tokenizer.save_pretrained(quant_name)

Repo card metadata block was not found. Setting CardData to empty.
AWQ: 100%|██████████| 32/32 [2:00:32<00:00, 226.01s/it]


('StarkWizard/Mistral-7b-instruct-cairo-instruct-AWQ/tokenizer_config.json',
 'StarkWizard/Mistral-7b-instruct-cairo-instruct-AWQ/special_tokens_map.json',
 'StarkWizard/Mistral-7b-instruct-cairo-instruct-AWQ/tokenizer.model',
 'StarkWizard/Mistral-7b-instruct-cairo-instruct-AWQ/added_tokens.json')

In [16]:
from huggingface_hub import HfApi

api = HfApi()

api.upload_folder(
    folder_path=quant_name,
    repo_id=quant_name,
    repo_type="model",
)

model-00002-of-00005.safetensors:   0%|          | 0.00/989M [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/907M [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/998M [00:00<?, ?B/s]

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/994M [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/262M [00:00<?, ?B/s]

'https://huggingface.co/StarkWizard/Mistral-7b-instruct-cairo-instruct-AWQ/tree/main/'

In [13]:
#Pushing Files to HF

from huggingface_hub import HfApi

api = HfApi()
repo = quant_name
local = "./"+repo+"/"
files =[
    local+"config.json",
    local+"generation_config.json",
    local+"quant_config.json",
    local+"tokenizer_config.json",
    local+"special_tokens_map.json",
] 

for f in files:
    name = f.split("/")[-1]

    api.upload_file(
        path_or_fileobj=f,
        path_in_repo=name,
        repo_id=repo,
        repo_type="model",
    )
    print(f"Uploaded {name}")

Uploaded config.json
Uploaded generation_config.json
Uploaded quant_config.json
Uploaded tokenizer_config.json
Uploaded special_tokens_map.json


ValueError: Provided path: 'StarkWizard/Mistral-7b-instruct-cairo-instruct-AWQ/tokenizer.json' is not a file on the local file system

In [ ]:
model= AutoAWQForCausalLM.from_quantized(quant_name,device_map="auto",fuse_layers=True,
                trust_remote_code=True,safetensors=True)

tokenizer = AutoTokenizer.from_pretrained(quant_name,trust_remote_code=True, use_auth_token=True, add_eos_token=True, use_fast=False)